# Coronavirus Data Acquisition

In [2]:
import pandas as pd
import urllib.request as urllib2
from bs4 import BeautifulSoup
import calendar
from datetime import datetime

In [3]:
def download_covid_data():
    """Downloads COVID19 Daily numbers of new cases and new deaths around the world"""
    url = 'https://covid.ourworldindata.org/data/ecdc/full_data.csv'
    response = urllib2.urlopen(url)
    df = pd.read_csv(response)
    df = df[df['location'] == 'World']
    return df

In [4]:
def add_prev_dates(df):
    """Captures only the most recent 2 years of COVID19 data, March X 2018 -> March X 2020"""
    c = calendar.Calendar()
    
    #generate new dates using calendar library
    years = [2018, 2019, 2020]
    dates = []
    for year in years:
        for i in range(1,13):
            for day in c.itermonthdates(year, i):
                if day not in dates:
                    dates.append(day)
    #capture most receent date            
    last_date = df['date'].iloc[-1]
    
    #clean dates list by cutting down dates to only March X 2018 -> March X 2020
    #slicing by most recent date
    for day in dates:
        if str(day) == last_date:
            end_date = dates.index(day) + 1
            dates= dates[:end_date]
            #reverse list, count to 730 and slice
            dates = dates[::-1]
            dates = dates[:754]
            dates = dates[::-1]
     #slicing by beginning date
    for day in dates:
        if str(day) == df['date'].iloc[0]:
            end_slice = dates.index(day)
            dates = dates[:end_slice]
            
    #add to dataframe and fill with 0s
    df_new = pd.DataFrame(dates, columns=['date'])
    df_new = df_new.append(df, sort=True)
    df_new['date'] = df_new['date'].astype('str')
    df_new = df_new.fillna(0)
    return df_new

In [5]:
def date_to_timestamp(col):
    """evaluates string date values and creates index of corresponding UNIX timestamps, preserves OG date"""
    #clean
    b = [val.split('-') for val in df[col]]
    c = []
    for date in b:
        a = []
        for num in date:
            a.append(int(num))
        c.append(a)
    
    #change tuples to Unix timestampls
    unix = []
    for val in c:
        date = datetime(val[0], val[1], val[2])
        unix.append(datetime.timestamp(date))
    df['UNIX'] = unix
    
    return df

In [6]:
def clean_df(df):
    df = df.set_index('UNIX')
    df = df.drop(columns={'total_cases', 'total_deaths', 'location'})
    return df

In [7]:
df = download_covid_data()
df

,date,location,new_cases,new_deaths,total_cases,total_deaths
11089,2019-12-31,World,27,0,27,0
11090,2020-01-01,World,0,0,27,0
11091,2020-01-02,World,0,0,27,0
11092,2020-01-03,World,17,0,44,0
11093,2020-01-04,World,0,0,44,0
...,...,...,...,...,...,...
11192,2020-04-12,World,81707,6104,1734913,108192
11193,2020-04-13,World,72390,5321,1807303,113513
11194,2020-04-14,World,65962,5339,1873265,118852
11195,2020-04-15,World,75246,7114,1948511,125966


In [8]:
df = add_prev_dates(df)
df

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2018-03-25,0,0.0,0.0,0.0,0.0
1,2018-03-26,0,0.0,0.0,0.0,0.0
2,2018-03-27,0,0.0,0.0,0.0,0.0
3,2018-03-28,0,0.0,0.0,0.0,0.0
4,2018-03-29,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11192,2020-04-12,World,81707.0,6104.0,1734913.0,108192.0
11193,2020-04-13,World,72390.0,5321.0,1807303.0,113513.0
11194,2020-04-14,World,65962.0,5339.0,1873265.0,118852.0
11195,2020-04-15,World,75246.0,7114.0,1948511.0,125966.0


In [9]:
df = date_to_timestamp('date')

In [10]:
df = clean_df(df)

In [11]:
df = df.head(-1)

In [12]:
df.to_csv('coviddata_april9.csv')

# Finance Data

In [53]:
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
start = dt.datetime(2018, 3, 19)
end = dt.datetime(2020, 3, 19)
stocks = web.DataReader(['EURUSD=X','PL=F',"GC=F",'CL=F',"SI=F","ETHUSD=X","JY=F","^DJI"], 'yahoo', start, end)["Close"]
stocks2 = web.DataReader(["BTC-USD"], 'yahoo', start, end)["Close"]
finadata = pd.merge(stocks,stocks2, on ="Date")
finadata.reset_index
finadata =  finadata.rename(columns={"EURUSD=X":"EURO", "PL=F":"Platinum",'CL=F':"Crude Oil",
                                    "SI=F":"Silver","ETHUSD=X":"Ethereum","JY=F": "Yeng","BTC-USD":"BITCOIN","GC=F":"Gold" })

In [54]:
finadata.tail(11)

Symbols,EURO,Platinum,Gold,Crude Oil,Silver,Ethereum,Yeng,^DJI,BITCOIN
Date,,,,,,,,,
2020-03-09,1.138602,896.400024,1672.400024,41.279999,17.214001,201.986328,0.009720,23851.019531,7923.644531
2020-03-10,1.139796,862.799988,1675.699951,31.129999,17.000999,200.767258,0.009520,25018.160156,7909.729492
2020-03-11,1.131651,869.400024,1660.300049,34.360001,16.906000,194.868515,0.009564,23553.220703,7911.430176
2020-03-12,1.126152,868.200012,1642.300049,32.980000,16.729000,112.347130,0.009555,21200.619141,4970.788086
2020-03-13,1.118493,763.299988,1590.300049,32.930000,14.685000,133.201813,0.009306,23185.619141,5563.707031
2020-03-15,NaN,756.900024,1515.699951,30.410000,14.765000,NaN,0.009471,NaN,5392.314941
2020-03-16,1.115536,743.900024,1516.699951,31.730000,14.500000,110.605873,0.009477,20188.519531,5014.479980
2020-03-17,1.116321,669.700012,1539.500000,26.440001,12.755000,113.942757,0.009357,21237.380859,5225.629395
2020-03-18,1.101552,646.099976,1500.099976,23.600000,12.270000,114.842270,0.009279,19898.919922,5238.438477


In [55]:
def datetime_to_string(dt):
    dtStr = dt.strftime('%Y.%m.%d')
    return str(dtStr)

finadata.index = finadata.index.map(datetime_to_string)

In [56]:
def reformat_datetime(dtStr):
    splitStr = dtStr.split(".")
    year = str(splitStr[0])
    month = str(splitStr[1]).lstrip("0")
    day = str(splitStr[2]).lstrip("0")
    
    return "[" + year + ", " + month + ", " + day + "]"

finadata.index = finadata.index.map(reformat_datetime)

In [57]:

finadata["Date"] = finadata.index
finadata.to_csv("temp_finadata.csv", index=False)
temp = pd.read_csv("temp_finadata.csv")
temp.tail(20)

,EURO,Platinum,Gold,Crude Oil,Silver,Ethereum,Yeng,^DJI,BITCOIN,Date
607,1.088589,904.299988,1643.099976,46.810001,17.799999,226.753387,0.009130,25766.640625,8784.494141,"[2020, 2, 27]"
608,1.099723,905.500000,1642.500000,47.090000,17.658001,226.760498,0.009262,25409.359375,8672.455078,"[2020, 2, 28]"
609,NaN,852.400024,1564.099976,44.570000,16.535000,NaN,0.009293,NaN,8562.454102,"[2020, 3, 1]"
610,1.105131,864.700012,1566.699951,44.759998,16.386999,230.569794,0.009241,26703.320312,8869.669922,"[2020, 3, 2]"
611,1.114405,859.400024,1594.800049,46.750000,16.679001,224.479630,0.009355,25917.410156,8787.786133,"[2020, 3, 3]"
612,1.118068,869.299988,1644.400024,47.180000,17.129000,224.517975,0.009296,27090.859375,8755.246094,"[2020, 3, 4]"
613,1.113908,875.200012,1643.000000,46.779999,17.187000,229.268188,0.009422,26121.279297,9078.762695,"[2020, 3, 5]"
614,1.122385,865.700012,1668.000000,45.900002,17.341999,243.525314,0.009497,25864.779297,9122.545898,"[2020, 3, 6]"
615,NaN,896.400024,1670.800049,32.540001,17.530001,NaN,0.009620,NaN,8108.116211,"[2020, 3, 8]"
616,1.138602,896.400024,1672.400024,41.279999,17.214001,201.986328,0.009720,23851.019531,7923.644531,"[2020, 3, 9]"


In [58]:
corona = pd.read_csv("corona_cases.csv")
corona.set_index("Date")
corona.tail(20)

,Date,new_cases,new_deaths
712,"[2020, 2, 29]",1838,64
713,"[2020, 3, 1]",1821,58
714,"[2020, 3, 2]",2044,67
715,"[2020, 3, 3]",1595,72
716,"[2020, 3, 4]",2413,84
717,"[2020, 3, 5]",2239,80
718,"[2020, 3, 6]",2856,103
719,"[2020, 3, 7]",3961,103
720,"[2020, 3, 8]",3691,97
721,"[2020, 3, 9]",3871,229


In [59]:
final_df = pd.merge(temp, corona, on="Date")
final_df.tail(40)

,EURO,Platinum,Gold,Crude Oil,Silver,Ethereum,Yeng,^DJI,BITCOIN,Date,new_cases,new_deaths
586,1.108979,968.000000,1577.199951,50.110001,17.639999,189.865067,0.009209,28399.810547,9293.521484,"[2020, 2, 3]",2818,57
587,1.106293,963.400024,1550.400024,49.610001,17.528000,189.250610,0.009138,28807.630859,9180.962891,"[2020, 2, 4]",3243,65
588,1.104728,984.400024,1557.800049,50.750000,17.569000,204.230240,0.009111,29290.849609,9613.423828,"[2020, 2, 5]",3907,66
589,1.100231,964.700012,1565.099976,50.950001,17.789000,212.339081,0.009098,29379.769531,9729.801758,"[2020, 2, 6]",3751,72
590,1.098298,966.599976,1568.599976,50.320000,17.665001,222.726074,0.009117,29102.509766,9795.943359,"[2020, 2, 7]",3218,73
591,NaN,970.200012,1577.300049,49.759998,17.719999,NaN,0.009137,NaN,10116.673828,"[2020, 2, 9]",2619,89
592,1.094931,964.299988,1574.699951,49.570000,17.757999,223.522705,0.009119,29276.820312,9856.611328,"[2020, 2, 10]",2988,97
593,1.091346,970.900024,1565.599976,49.939999,17.573000,235.851196,0.009115,29276.339844,10208.236328,"[2020, 2, 11]",2565,108
594,1.091941,964.500000,1567.400024,51.169998,17.478001,265.406128,0.009087,29551.419922,10326.054688,"[2020, 2, 12]",2072,97
595,1.087595,971.900024,1575.099976,51.419998,17.600000,268.099274,0.009108,29423.310547,10214.379883,"[2020, 2, 13]",15151,255


In [62]:
final_df.to_csv("final_df.csv", index=False)

In [45]:
df

,date,new_cases,new_deaths
UNIX,,,
1.521432e+09,2018-03-19,0.0,0.0
1.521518e+09,2018-03-20,0.0,0.0
1.521605e+09,2018-03-21,0.0,0.0
1.521691e+09,2018-03-22,0.0,0.0
1.521778e+09,2018-03-23,0.0,0.0
...,...,...,...
1.586059e+09,2020-04-05,86713.0,6115.0
1.586146e+09,2020-04-06,71232.0,4655.0
1.586232e+09,2020-04-07,71390.0,5137.0
